In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

In [2]:
KEY = 'cd17e816e9504445b9ce76b66e4bab18'
ENDPOINT = 'https://anonfaces.cognitiveservices.azure.com/'

In [3]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [35]:
# Used in the Person Group Operations,  Snapshot Operations, and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
PERSON_GROUP_ID = 'hale-x20-x89'

# Used for the Snapshot and Delete Person Group examples.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

In [36]:
'''
Create the PersonGroup
'''
# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Define woman friend
woman = face_client.person_group_person.create(PERSON_GROUP_ID, "Woman")
# Define man friend
man = face_client.person_group_person.create(PERSON_GROUP_ID, "Man")
# Define child friend
child = face_client.person_group_person.create(PERSON_GROUP_ID, "Child")


Person group: hale-x20-x89


In [39]:
woman.person_id

'87674458-301c-41f4-a94f-45341b96f381'

In [46]:
'''
Detect faces and register to correct person
'''
data_dir = 'Data/'
# Find all jpeg images of friends in working directory
woman_images = [data_dir + file for file in os.listdir('Data/') if file.startswith("woman")]
man_images = [data_dir + file for file in os.listdir('Data/') if file.startswith("man")]
child_images = [data_dir + file for file in os.listdir('Data/') if file.startswith("child")]

# Add to a woman person
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, woman.person_id, w)

# Add to a man person
for image in man_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, man.person_id, m)

# Add to a child person
for image in child_images:
    ch = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)


In [44]:
glob.glob('Data/*.jpg')

['Data/man3-person-group.jpg',
 'Data/woman3-person-group.jpg',
 'Data/woman1-person-group.jpg',
 'Data/child3-person-group.jpg',
 'Data/woman2-person-group.jpg',
 'Data/child2-person-group.jpg',
 'Data/man1-person-group.jpg',
 'Data/test-image-person-group.jpg',
 'Data/man2-person-group.jpg',
 'Data/extra-woman-image.jpg',
 'Data/child1-person-group.jpg']

In [47]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: running.

Training status: succeeded.



In [48]:
'''
Identify a face against a defined PersonGroup
'''
# Group image for testing against
group_photo = 'test-image-person-group.jpg'
IMAGES_FOLDER = 'Data/'
test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, group_photo))
image = open(test_image_array[0], 'r+b')

# Detect faces
face_ids = []
faces = face_client.face.detect_with_stream(image)
for face in faces:
    face_ids.append(face.face_id)

In [49]:
# Identify faces
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
print('Identifying faces in {}'.format(os.path.basename(image.name)))
if not results:
    print('No person identified in the person group for faces from {}.'.format(os.path.basename(image.name)))
for person in results:
    print('Person with ID {} is identified in {} with a confidence of {}.'.format(person.candidates[0].person_id, os.path.basename(image.name), person.candidates[0].confidence)) # Get topmost confidence score

Identifying faces in test-image-person-group.jpg
Person with ID d06e89ce-66ae-4ee8-9bb0-bd95b1499eb4 is identified in test-image-person-group.jpg with a confidence of 0.92387.
Person with ID 87674458-301c-41f4-a94f-45341b96f381 is identified in test-image-person-group.jpg with a confidence of 0.93316.


In [27]:
results[1].candidates[0].person_id

'4dbbbc26-d89a-47a8-b812-9dc7b7f271e5'

In [55]:
print(woman.as_dict())

<bound method Model.as_dict of <azure.cognitiveservices.vision.face.models._models_py3.Person object at 0x7f5f42be0828>>


In [50]:
print('woman: ', woman.person_id)
print('child: ', child.person_id)
print('man: ', man.person_id)

woman:  87674458-301c-41f4-a94f-45341b96f381
child:  c3faff3c-6adc-4551-830a-e5ea1ae988c4
man:  d06e89ce-66ae-4ee8-9bb0-bd95b1499eb4


In [33]:
face_client.person_group_person.list(PERSON_GROUP_ID)[0].